## Module 2 Homework (2025 Cohort)

In this homework, we're going to combine data from various sources to process it in Pandas and generate additional fields.

If not stated otherwise, please use the [LINK](https://github.com/DataTalksClub/stock-markets-analytics-zoomcamp/blob/main/02-dataframe-analysis/%5B2025%5D_Module_02_Colab_Working_with_the_data.ipynb) covered at the livestream to re-use the code snippets.

---
### Question 1: [IPO] Withdrawn IPOs by Company Type

**What is the total withdrawn IPO value (in $ millions) for the company class with the highest total withdrawal value?**

From the withdrawn IPO list ([stockanalysis.com/ipos/withdrawn](https://stockanalysis.com/ipos/withdrawn/)), collect and process the data to find out which company type saw the most withdrawn IPO value.

1. Use `pandas.read_html()` with the URL above to load the IPO withdrawal table into a DataFrame. 
   *It is a similar process to Code Snippet 1 discussed at the livestream.*    You should get **99 entries**. 

In [48]:
import pandas as pd
import requests
from io import StringIO
url = "https://stockanalysis.com/ipos/withdrawn/"
def get_table(url) -> pd.DataFrame:
    """
    Fetch IPO data for the given year from stockanalysis.com.
    """
   
    headers = {
        'User-Agent': (
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
            'AppleWebKit/537.36 (KHTML, like Gecko) '
            'Chrome/58.0.3029.110 Safari/537.3'
        )
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        # Wrap HTML text in StringIO to avoid deprecation warning
        # "Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object."
        html_io = StringIO(response.text)
        tables = pd.read_html(html_io)

        if not tables:
            raise ValueError(f"No tables found for year {year}.")

        return tables[0]

    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    except ValueError as ve:
        print(f"Data error: {ve}")
    except Exception as ex:
        print(f"Unexpected error: {ex}")

    return pd.DataFrame()

In [49]:
ipos_df= get_table(url)
ipos_df

,Symbol,Company Name,Price Range,Shares Offered
0,ODTX,"Odyssey Therapeutics, Inc.",-,-
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000
2,AURN,"Aurion Biotech, Inc.",-,-
3,ROTR,"PHI Group, Inc.",-,-
4,ONE,One Power Company,-,-
...,...,...,...,...
95,FHP,"Freehold Properties, Inc.",-,-
96,CHO,Chobani Inc.,-,-
97,IFIT,iFIT Health & Fitness Inc.,$18.00 - $21.00,30769231
98,GLGX,"Gerson Lehrman Group, Inc.",-,-


2. Create a new column called `Company Class`, categorizing company names based on patterns like:
   - “Acquisition Corp” or “Acquisition Corporation” → `Acq.Corp`
   - “Inc” or “Incorporated” → `Inc`
   - “Group” → `Group`
   - “Ltd” or “Limited” → `Limited`
   - “Holdings” → `Holdings`
   - Others → `Other`

  *  Order: Please follow the listed order of classes and assign the first matched value (e.g., for 'shenni holdings limited', you assign the 'Limited' class).

  * Hint: make your function more robust by converting names to lowercase and splitting into words before matching patterns.

In [50]:
import pandas as pd
import yfinance as yf



def classify_company(name):
    words = name.lower().split()

    if  words[-1]== "inc." or words[-1]== "incorporated" :
        return "Inc"
    if  words[-1]== "ltd" or   words[-1]== "ltd." or words[-1]== "limited" : 
    #if 'inc' in words or 'incorporated' in words:
        return 'LTD'
    if  words[-1]== "corp." or   words[-1]== "corporation" : 
    #if 'inc' in words or 'incorporated' in words:
        return 'CORP'
    if  words[-1]== "group." or   words[-1]== "group" : 
    #if 'inc' in words or 'incorporated' in words:
        return 'group' 
    if  words[-1]== "holdings." or   words[-1]== "holdings" : 
    #
        return 'Holdings'        
    else:
        return "other"




ipos_df["Company Class"] = ipos_df["Company Name"].apply(classify_company)
ipos_df

,Symbol,Company Name,Price Range,Shares Offered,Company Class
0,ODTX,"Odyssey Therapeutics, Inc.",-,-,Inc
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000,Inc
2,AURN,"Aurion Biotech, Inc.",-,-,Inc
3,ROTR,"PHI Group, Inc.",-,-,Inc
4,ONE,One Power Company,-,-,other
...,...,...,...,...,...
95,FHP,"Freehold Properties, Inc.",-,-,Inc
96,CHO,Chobani Inc.,-,-,Inc
97,IFIT,iFIT Health & Fitness Inc.,$18.00 - $21.00,30769231,Inc
98,GLGX,"Gerson Lehrman Group, Inc.",-,-,Inc


3. Define a new field `Avg. price` by parsing the `Price Range` field (create a function and apply it to the `Price Range` column). Examples:
   - '$8.00-$10.00' → `9.0`  
   - '$5.00' → `5.0`  
   - '-' → `None`

In [51]:
def parse_avg_price(price_str):
    # Remove $ signs and whitespace
    price_str = price_str.replace('$', '').strip()
    
    if price_str == '-' or price_str == '':
        return None
    
    try:
        if '-' in price_str:
            parts = price_str.split('-')
            prices = [float(p.strip()) for p in parts]
            return sum(prices) / len(prices)
        else:
            return float(price_str)
    except:
        return None  # Fallback for malformed data

# Apply it to your DataFrame
ipos_df['Avg. price'] = ipos_df['Price Range'].apply(parse_avg_price)


In [52]:
ipos_df

,Symbol,Company Name,Price Range,Shares Offered,Company Class,Avg. price
0,ODTX,"Odyssey Therapeutics, Inc.",-,-,Inc,NaN
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000,Inc,3.5
2,AURN,"Aurion Biotech, Inc.",-,-,Inc,NaN
3,ROTR,"PHI Group, Inc.",-,-,Inc,NaN
4,ONE,One Power Company,-,-,other,NaN
...,...,...,...,...,...,...
95,FHP,"Freehold Properties, Inc.",-,-,Inc,NaN
96,CHO,Chobani Inc.,-,-,Inc,NaN
97,IFIT,iFIT Health & Fitness Inc.,$18.00 - $21.00,30769231,Inc,19.5
98,GLGX,"Gerson Lehrman Group, Inc.",-,-,Inc,NaN


4. Convert `Shares Offered` to numeric, clean missing or invalid values.


In [53]:
ipos_df['Shares Offered'] = pd.to_numeric(ipos_df['Shares Offered'], errors='coerce')


In [54]:
ipos_df

,Symbol,Company Name,Price Range,Shares Offered,Company Class,Avg. price
0,ODTX,"Odyssey Therapeutics, Inc.",-,NaN,Inc,NaN
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000.0,Inc,3.5
2,AURN,"Aurion Biotech, Inc.",-,NaN,Inc,NaN
3,ROTR,"PHI Group, Inc.",-,NaN,Inc,NaN
4,ONE,One Power Company,-,NaN,other,NaN
...,...,...,...,...,...,...
95,FHP,"Freehold Properties, Inc.",-,NaN,Inc,NaN
96,CHO,Chobani Inc.,-,NaN,Inc,NaN
97,IFIT,iFIT Health & Fitness Inc.,$18.00 - $21.00,30769231.0,Inc,19.5
98,GLGX,"Gerson Lehrman Group, Inc.",-,NaN,Inc,NaN


5. Create a new column:  
   `Withdrawn Value = Shares Offered * Avg Price` (**71 non-null values**)

In [55]:
# Ensure Shares Offered and Avg. price are numeric
df=ipos_df
df['Shares Offered'] = pd.to_numeric(df['Shares Offered'], errors='coerce')
df['Avg. price'] = pd.to_numeric(df['Avg. price'], errors='coerce')

# Create the Withdrawn Value column
df['Withdrawn Value'] = df['Shares Offered'] * df['Avg. price']


In [56]:
non_null_count = df['Withdrawn Value'].notnull().sum()
print(f"Non-null Withdrawn Value entries: {non_null_count}")


Non-null Withdrawn Value entries: 71


In [57]:
df = df.dropna(subset=['Withdrawn Value'])
df

,Symbol,Company Name,Price Range,Shares Offered,Company Class,Avg. price,Withdrawn Value
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000.0,Inc,3.5,7000000.0
5,HPOT,The Great Restaurant Development Holdings Limited,$4.00 - $6.00,1400000.0,LTD,5.0,7000000.0
6,CABR,"Caring Brands, Inc.",$4.00,750000.0,Inc,4.0,3000000.0
7,SQVI,"Sequoia Vaccines, Inc.",$8.00 - $10.00,2775000.0,Inc,9.0,24975000.0
8,SNI,Shenni Holdings Limited,$4.00 - $6.00,3000000.0,LTD,5.0,15000000.0
...,...,...,...,...,...,...,...
92,DPAC,Deep Space Acquisition Corp. I,$10.00,21000000.0,other,10.0,210000000.0
93,GIF,"GigCapital6, Inc.",$10.00,20000000.0,Inc,10.0,200000000.0
94,HYIV,Haymaker Acquisition Corp. IV,$10.00,26100000.0,other,10.0,261000000.0
97,IFIT,iFIT Health & Fitness Inc.,$18.00 - $21.00,30769231.0,Inc,19.5,600000004.5


6. Group by `Company Class` and calculate total withdrawn value.


In [58]:
# Group by Company Class and sum Withdrawn Value
withdrawn_by_class = df.groupby('Company Class')['Withdrawn Value'].sum().reset_index()
withdrawn_by_class

,Company Class,Withdrawn Value
0,CORP,2.265850e+09
1,Inc,2.257164e+09
2,LTD,7.313346e+08
3,group,2.718750e+07
4,other,2.423070e+09


7. **Answer**: Which class had the highest **total** value of withdrawals?
---

In [59]:

# Optional: Sort by highest total value
withdrawn_by_class = withdrawn_by_class.sort_values(by='Withdrawn Value', ascending=False)

print(withdrawn_by_class)

  Company Class  Withdrawn Value
4         other     2.423070e+09
0          CORP     2.265850e+09
1           Inc     2.257164e+09
2           LTD     7.313346e+08
3         group     2.718750e+07


### Question 2:   [IPO] Median Sharpe Ratio for 2024 IPOs (First 5 Months)


**What is the median Sharpe ratio (as of 6 June 2025) for companies that went public in the first 5 months of 2024?**

The goal is to replicate the large-scale `yfinance` OHLCV data download and perform basic financial calculations on IPO stocks.



1. Using the same approach as in Question 1, download the IPOs in 2024 from:  
   [https://stockanalysis.com/ipos/2024/](https://stockanalysis.com/ipos/2024/)  
   Filter to keep only those IPOs **before 1 June 2024** (first 5 months of 2024).  
   ➤ You should have **75 tickers**.

In [60]:
url="https://stockanalysis.com/ipos/2024/"
ipos_df2024= get_table(url)
ipos_df2024

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,"Dec 31, 2024",ONEG,OneConstruction Group Limited,$4.00,$5.00,25.00%
1,"Dec 27, 2024",PHH,"Park Ha Biological Technology Co., Ltd.",$4.00,$26.57,564.25%
2,"Dec 23, 2024",HIT,"Health In Tech, Inc.",$4.00,$0.62,-84.40%
3,"Dec 23, 2024",TDAC,Translational Development Acquisition Corp.,$10.00,$10.28,2.80%
4,"Dec 20, 2024",RANG,Range Capital Acquisition Corp.,$10.00,$10.21,2.10%
...,...,...,...,...,...,...
220,"Jan 18, 2024",CCTG,CCSC Technology International Holdings Limited,$6.00,$1.07,-82.17%
221,"Jan 18, 2024",PSBD,Palmer Square Capital BDC Inc.,$16.45,$14.50,-11.85%
222,"Jan 12, 2024",SYNX,Silynxcom Ltd.,$4.00,$2.07,-48.25%
223,"Jan 11, 2024",SDHC,Smith Douglas Homes Corp.,$21.00,$18.26,-13.05%


In [61]:
ipos_df2024["IPO Date"]

0      Dec 31, 2024
1      Dec 27, 2024
2      Dec 23, 2024
3      Dec 23, 2024
4      Dec 20, 2024
           ...     
220    Jan 18, 2024
221    Jan 18, 2024
222    Jan 12, 2024
223    Jan 11, 2024
224     Jan 9, 2024
Name: IPO Date, Length: 225, dtype: object

In [62]:
def filter_first_5_months_str(date_series):
    """
    Filter dates whose string starts with Jan, Feb, Mar, Apr, or May.
    Expects a pd.Series of strings.
    Returns a boolean mask.
    """
    valid_months = ['Jan', 'Feb', 'Mar', 'Apr', 'May']
    return date_series.str.startswith(tuple(valid_months))

# Usage:
df=ipos_df2024
df=df[df["IPO Price"]!="-"]
filtered_df = df[filter_first_5_months_str(df['IPO Date'])]

print(filtered_df.shape[0])  # number of rows where IPO Date is Jan-May


75


In [63]:
df=ipos_df2024
df=df[df["IPO Price"]=="-"]
df

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
61,"Oct 11, 2024",STFS,Star Fashion Culture Holdings Limited,-,$1.14,-
120,"Jul 23, 2024",ZDAI,Primega Group Holdings Limited,-,$0.88,-
148,"May 31, 2024",NAKA,"Kindly MD, Inc.",-,$14.22,-
174,"Apr 17, 2024",SUPX,SuperX AI Technology Limited,-,$9.79,-


In [64]:
filtered_df["Symbol"].to_list()

['BOW',
 'HDL',
 'RFAI',
 'JDZG',
 'RAY',
 'BTOC',
 'ZK',
 'GPAT',
 'PAL',
 'SVCO',
 'NNE',
 'CCIX',
 'VIK',
 'ZONE',
 'LOAR',
 'MRX',
 'RBRK',
 'NCI',
 'MFI',
 'YYGH',
 'TRSG',
 'CDTG',
 'CTRI',
 'IBTA',
 'MTEN',
 'TWG',
 'ULS',
 'PACS',
 'MNDR',
 'CTNM',
 'MAMO',
 'ZBAO',
 'BOLD',
 'MMA',
 'UBXG',
 'IBAC',
 'AUNA',
 'BKHA',
 'LOBO',
 'RDDT',
 'ALAB',
 'INTJ',
 'RYDE',
 'LGCL',
 'SMXT',
 'VHAI',
 'DYCQ',
 'CHRO',
 'UMAC',
 'HLXB',
 'MGX',
 'TBBB',
 'TELO',
 'KYTX',
 'PMNT',
 'AHR',
 'LEGT',
 'ANRO',
 'GUTS',
 'AS',
 'FBLG',
 'AVBP',
 'BTSG',
 'HAO',
 'CGON',
 'YIBO',
 'JL',
 'SUGP',
 'JVSA',
 'KSPI',
 'CCTG',
 'PSBD',
 'SYNX',
 'SDHC',
 'ROMA']

2.  Use **Code Snippet 7** to download daily stock data for those tickers (via `yfinance`).  
   Make sure you understand how `growth_1d` ... `growth_365d`, and volatility columns are defined.  
   Define a new column `growth_252d` representing growth after **252 trading days** (~1 year), in addition to any other growth periods you already track.


In [65]:
##### 
import numpy as np

ALL_TICKERS = filtered_df["Symbol"].to_list()
import time

stocks_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(ALL_TICKERS):
  print(i,ticker)

  # Work with stock prices
  ticker_obj = yf.Ticker(ticker)

  # historyPrices = yf.download(tickers = ticker,
  #                    period = "max",
  #                    interval = "1d")
  historyPrices = ticker_obj.history(
                     period = "max",
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = ticker
  historyPrices['Year']= historyPrices.index.year
  historyPrices['Month'] = historyPrices.index.month
  historyPrices['Weekday'] = historyPrices.index.weekday
  historyPrices['Date'] = historyPrices.index.date

  # historical returns
  for i in [1,3,7,30,90,252,365]:
    historyPrices['growth_'+str(i)+'d'] = historyPrices['Close'] / historyPrices['Close'].shift(i)
  historyPrices['growth_future_30d'] = historyPrices['Close'].shift(-30) / historyPrices['Close']

  # Technical indicators
  # SimpleMovingAverage 10 days and 20 days
  historyPrices['SMA10']= historyPrices['Close'].rolling(10).mean()
  historyPrices['SMA20']= historyPrices['Close'].rolling(20).mean()
  historyPrices['growing_moving_average'] = np.where(historyPrices['SMA10'] > historyPrices['SMA20'], 1, 0)
  historyPrices['high_minus_low_relative'] = (historyPrices.High - historyPrices.Low) / historyPrices['Close']

  # 30d rolling volatility : https://ycharts.com/glossary/terms/rolling_vol_30
  historyPrices['volatility'] =   historyPrices['Close'].rolling(30).std() * np.sqrt(252)

  # what we want to predict
  historyPrices['is_positive_growth_30d_future'] = np.where(historyPrices['growth_future_30d'] > 1, 1, 0)

  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)


  if stocks_df.empty:
    stocks_df = historyPrices
  else:
    stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)

0 BOW
1 HDL
2 RFAI
3 JDZG
4 RAY
5 BTOC
6 ZK
7 GPAT
8 PAL
9 SVCO
10 NNE
11 CCIX
12 VIK
13 ZONE
14 LOAR
15 MRX
16 RBRK
17 NCI
18 MFI
19 YYGH
20 TRSG
21 CDTG
22 CTRI
23 IBTA
24 MTEN
25 TWG
26 ULS
27 PACS
28 MNDR
29 CTNM
30 MAMO
31 ZBAO
32 BOLD
33 MMA
34 UBXG
35 IBAC
36 AUNA
37 BKHA
38 LOBO
39 RDDT
40 ALAB
41 INTJ
42 RYDE
43 LGCL
44 SMXT
45 VHAI
46 DYCQ
47 CHRO
48 UMAC
49 HLXB
50 MGX
51 TBBB
52 TELO
53 KYTX
54 PMNT
55 AHR
56 LEGT
57 ANRO
58 GUTS
59 AS
60 FBLG
61 AVBP
62 BTSG
63 HAO
64 CGON
65 YIBO
66 JL
67 SUGP
68 JVSA
69 KSPI
70 CCTG
71 PSBD
72 SYNX
73 SDHC
74 ROMA


In [66]:
stocks_df.Ticker

0         BOW
1         BOW
2         BOW
3         BOW
4         BOW
         ... 
23316    ROMA
23317    ROMA
23318    ROMA
23319    ROMA
23320    ROMA
Name: Ticker, Length: 23321, dtype: object

In [67]:

stocks_df['Date'] = pd.to_datetime(stocks_df['Date'])

In [68]:
for x in [1,3,7,30,90,252,365]:
    print(historyPrices["growth_"+str(x)+"d"].isna().sum(),len(historyPrices["growth_"+str(x)+"d"]))

1 363
3 363
7 363
30 363
90 363
252 363
363 363


In [69]:
stocks_df['growth_252d']

0             NaN
1             NaN
2             NaN
3             NaN
4             NaN
           ...   
23316    4.666667
23317    5.424529
23318    5.008960
23319    5.157116
23320    5.325670
Name: growth_252d, Length: 23321, dtype: float64


3. Calculate the Sharpe ratio assuming a risk-free rate of **4.5%**:

   ```python
   stocks_df['Sharpe'] = (stocks_df['growth_252d'] - 0.045) / stocks_df['volatility']
   ```

   ⚠️ **IMPORTANT** Please use the original version of annualized volatility calculation (it was later corrected to another formula):
   ```python
   stocks_df['volatility'] =   stocks_df['Close'].rolling(30).std() * np.sqrt(252)
   ```

In [70]:
stocks_df['Sharpe'] = (stocks_df['growth_252d'] - 0.045) / stocks_df['volatility']
stocks_df['volatility2'] =   stocks_df['Close'].rolling(30).std() * np.sqrt(252)
#historyPrices['volatility'] =   historyPrices['Close'].rolling(30).std() * np.sqrt(252)
stocks_df["volatility2"]

0              NaN
1              NaN
2              NaN
3              NaN
4              NaN
           ...    
23316    10.383358
23317     9.939668
23318     9.469241
23319     9.196931
23320     9.001998
Name: volatility2, Length: 23321, dtype: float64

In [71]:
stocks_df["volatility"]

0              NaN
1              NaN
2              NaN
3              NaN
4              NaN
           ...    
23316    10.383358
23317     9.939668
23318     9.469241
23319     9.196931
23320     9.001998
Name: volatility, Length: 23321, dtype: float64

4. Filter the DataFrame to keep data only for the trading day:  
   **‘2025-06-06’**

   Compute descriptive statistics (e.g., `.describe()`) for these columns:  
   - `growth_252d`  
   - `Sharpe`

   You should observe:  
   - `growth_252d` is defined for **71 out of 75 stocks** (some IPOs are too recent or data starts later).  
   - Median `growth_252d` is approximately **0.75** (indicating a 25% decline), while mean is about **1.15**, showing a bias towards high-growth companies pushing the average up.

In [72]:
ddd=stocks_df[stocks_df.Date=='2025-06-06']

In [73]:
ddd

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,Year,Month,...,growth_365d,growth_future_30d,SMA10,SMA20,growing_moving_average,high_minus_low_relative,volatility,is_positive_growth_30d_future,Sharpe,volatility2
259,36.439999,36.540001,35.930000,36.389999,137900,0.0,0.0,BOW,2025,6,...,NaN,NaN,36.9540,37.70400,0,0.016763,24.508713,0,0.057014,24.508713
532,20.430000,20.440001,19.969999,20.410000,1300,0.0,0.0,HDL,2025,6,...,NaN,NaN,19.9660,21.08215,0,0.023028,18.816306,0,0.051134,18.816306
773,10.510000,10.510000,10.510000,10.510000,0,0.0,0.0,RFAI,2025,6,...,NaN,NaN,10.4930,10.47935,1,0.000000,0.356043,0,NaN,0.356043
1048,0.380000,0.386000,0.201000,0.295000,3428900,0.0,0.0,JDZG,2025,6,...,NaN,NaN,0.4883,0.52880,0,0.627119,1.475159,0,0.083768,1.475159
1323,1.110000,1.270000,1.100000,1.255000,501000,0.0,0.0,RAY,2025,6,...,NaN,NaN,2.1015,1.90825,1,0.135458,9.314924,0,0.032041,9.314924
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21872,1.075000,1.100000,1.075000,1.095000,1600,0.0,0.0,CCTG,2025,6,...,NaN,NaN,1.1434,1.18080,0,0.022831,1.297728,0,0.350613,1.297728
22229,13.900000,14.000000,13.750000,13.880000,56200,0.0,0.0,PSBD,2025,6,...,NaN,NaN,13.9030,13.74250,1,0.018012,9.712591,0,0.092927,9.712591
22589,1.750000,1.750000,1.680000,1.680000,10000,0.0,0.0,SYNX,2025,6,...,NaN,NaN,1.7435,1.82090,0,0.041667,1.368371,0,0.425054,1.368371
22948,19.900000,20.150000,19.270000,19.270000,38100,0.0,0.0,SDHC,2025,6,...,NaN,NaN,18.6850,18.66500,1,0.045667,16.442821,0,0.043366,16.442821


In [74]:
ddd["growth_252d"].describe()

count    71.000000
mean      1.152897
std       1.406017
min       0.024970
25%       0.293422
50%       0.758065
75%       1.362736
max       8.097413
Name: growth_252d, dtype: float64

In [75]:
df_non_null = ddd[ddd["growth_252d"].notna()]


In [76]:
df_non_null["Sharpe"].describe()

count    71.000000
mean      0.288285
std       0.519028
min      -0.079677
25%       0.041215
50%       0.083768
75%       0.311507
max       2.835668
Name: Sharpe, dtype: float64

In [77]:
ddd["Sharpe"].describe()

count    71.000000
mean      0.288285
std       0.519028
min      -0.079677
25%       0.041215
50%       0.083768
75%       0.311507
max       2.835668
Name: Sharpe, dtype: float64

5. **Answer:**  
   - What is the **median Sharpe ratio** for these 71 stocks?  
   - Note: Positive `Sharpe` means growth exceeding the risk-free rate of 4.5%.  
   - [Additional] Do you observe the **same top 10 companies** when sorting by `growth_252d` versus sorting by `Sharpe`?

---

In [78]:
# Top 10 by growth
top_growth = ddd.sort_values(by='growth_252d', ascending=False).head(10)

# Top 10 by Sharpe ratio
top_sharpe = ddd.sort_values(by='Sharpe', ascending=False).head(10)

# Compare overlap
common = set(top_growth['Ticker']) & set(top_sharpe['Ticker'])

print("Number of companies in both top 10 lists:", len(common))
print("Common companies:", common)


Number of companies in both top 10 lists: 1
Common companies: {'JL'}


### Question 3: [IPO] ‘Fixed Months Holding Strategy’

**What is the optimal number of months (1 to 12) to hold a newly IPO'd stock in order to maximize average growth?**  
(*Assume you buy at the close of the first trading day and sell after a fixed number of trading days.*)


---

#### Goal:
Investigate whether holding an IPO stock for a fixed number of months after its first trading day produces better returns, using future growth columns.

---

1. **Start from the existing DataFrame** from Question 2 (75 tickers from IPOs in the first 5 months of 2024).  

   Add **12 new columns**:  
   `future_growth_1m`, `future_growth_2m`, ..., `future_growth_12m`  
   *(Assume 1 month = 21 trading days, so growth is calculated over 21, 42, ..., 252 trading days)*  
   This logic is similar to `historyPrices['growth_future_30d']` from **Code Snippet 7**, but extended to longer timeframes.

In [79]:
#GOOD#
stocks_df.shape[0]

23321

In [80]:
for i in [1,2,3,4,5,6,7,8,9,10,11,12]:
    stocks_df['future_growth_'+str(i)+'m'] = stocks_df['Close'] / stocks_df['Close'].shift(i*21)

2. **Determine the first trading day** (`min_date`) for each ticker.  
   This is the earliest date in the data for each stock.

In [81]:
# Make sure 'Date' column is in datetime format
stocks_df['Date'] = pd.to_datetime(stocks_df['Date'])

# Group by 'Ticker' and find the earliest date
min_dates = stocks_df.groupby('Ticker')['Date'].min().reset_index()

# Rename for clarity
min_dates.columns = ['Ticker', 'min_date']

# Display result
print(min_dates.head())



  Ticker   min_date
0    AHR 2024-02-07
1   ALAB 2024-03-20
2   ANRO 2024-02-02
3     AS 2024-02-01
4   AUNA 2024-03-22


In [82]:
stocks_df.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits',
       'Ticker', 'Year', 'Month', 'Weekday', 'Date', 'growth_1d', 'growth_3d',
       'growth_7d', 'growth_30d', 'growth_90d', 'growth_252d', 'growth_365d',
       'growth_future_30d', 'SMA10', 'SMA20', 'growing_moving_average',
       'high_minus_low_relative', 'volatility',
       'is_positive_growth_30d_future', 'Sharpe', 'volatility2',
       'future_growth_1m', 'future_growth_2m', 'future_growth_3m',
       'future_growth_4m', 'future_growth_5m', 'future_growth_6m',
       'future_growth_7m', 'future_growth_8m', 'future_growth_9m',
       'future_growth_10m', 'future_growth_11m', 'future_growth_12m'],
      dtype='object')

3. **Join the data**:  
   Perform an **inner join** between the `min_date` DataFrame and the future growth data on both `ticker` and `date`.  
   ➤ You should end up with **75 records** (one per IPO) with all 12 `future_growth_...` fields populated.

In [83]:
# Ensure date columns are datetime
min_dates['min_date'] = pd.to_datetime(min_dates['min_date'])
future_growth_df=stocks_df

# Inner join on Ticker and Date (first trading date)
merged_df = pd.merge(
    min_dates,
    future_growth_df,
    how='inner',
    left_on=['Ticker', 'min_date'],
    right_on=['Ticker', 'Date']
)

# Drop duplicate 'Date' column if needed
merged_df.drop(columns='Date', inplace=True)

# Rename min_date for clarity (optional)
merged_df.rename(columns={'min_date': 'Date'}, inplace=True)

# Check result
print(f"Resulting records: {len(merged_df)}") #75 rows --GOOOD
print(merged_df.head())


Resulting records: 75
  Ticker       Date       Open       High        Low      Close    Volume  \
0    AHR 2024-02-07  12.085785  12.471402  11.878869  12.433781  12732800   
1   ALAB 2024-03-20  52.560001  63.500000  50.610001  62.029999  16843300   
2   ANRO 2024-02-02  22.000000  23.270000  20.000000  20.700001   2386300   
3     AS 2024-02-01  13.400000  13.800000  13.100000  13.400000  18656400   
4   AUNA 2024-03-22   9.510000  10.320000   9.300000   9.600000   9046900   

   Dividends  Stock Splits  Year  ...  future_growth_3m  future_growth_4m  \
0        0.0           0.0  2024  ...         11.003346         12.189981   
1        0.0           0.0  2024  ...          0.536128          0.353568   
2        0.0           0.0  2024  ...          1.978968          1.994220   
3        0.0           0.0  2024  ...         10.720000          9.178082   
4        0.0           0.0  2024  ...          0.932039          0.938417   

   future_growth_5m  future_growth_6m  future_growth

4. **Compute descriptive statistics** for the resulting DataFrame:  
   Use `.describe()` or similar to analyze each of the 12 columns:  
   - `future_growth_1m`  
   - `future_growth_2m`  
   - ...  
   - `future_growth_12m`  

In [84]:
for i in [1,2,3,4,5,6,7,8,9,10,11,12]:
    print('future_growth_'+str(i)+'m')
    print(stocks_df['future_growth_'+str(i)+'m'].describe())

future_growth_1m
count    23300.000000
mean        17.575470
std        539.458797
min          0.004682
25%          0.846154
50%          0.996344
75%          1.097262
max      25300.000353
Name: future_growth_1m, dtype: float64
future_growth_2m
count    23279.000000
mean        29.742199
std        676.254347
min          0.004328
25%          0.741442
50%          0.981132
75%          1.155362
max      25425.000833
Name: future_growth_2m, dtype: float64
future_growth_3m
count    23258.000000
mean        37.650665
std        719.925130
min          0.004274
25%          0.650000
50%          0.966717
75%          1.232405
max      25500.000167
Name: future_growth_3m, dtype: float64
future_growth_4m
count    23237.000000
mean        42.683224
std        736.940889
min          0.003846
25%          0.563140
50%          0.960204
75%          1.320957
max      25550.001313
Name: future_growth_4m, dtype: float64
future_growth_5m
count    23216.000000
mean        45.484761
std        

In [85]:
stocks_df

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,Year,Month,...,future_growth_3m,future_growth_4m,future_growth_5m,future_growth_6m,future_growth_7m,future_growth_8m,future_growth_9m,future_growth_10m,future_growth_11m,future_growth_12m
0,23.000000,24.270000,22.139999,23.799999,3335800,0.0,0.0,BOW,2024,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,24.260000,26.150000,23.980000,25.700001,990500,0.0,0.0,BOW,2024,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,25.850000,26.879999,25.075001,26.480000,555100,0.0,0.0,BOW,2024,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,26.440001,26.490000,25.500999,26.290001,302700,0.0,0.0,BOW,2024,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,27.209999,27.209999,25.500000,26.139999,200900,0.0,0.0,BOW,2024,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23316,2.870000,2.890000,2.560000,2.660000,123100,0.0,0.0,ROMA,2025,6,...,3.419023,4.586207,4.018127,3.022727,2.686869,2.891304,4.570447,5.086042,4.360656,4.666667
23317,2.840000,3.000000,2.640000,2.875000,63100,0.0,0.0,ROMA,2025,6,...,3.639240,4.215543,4.166667,3.274487,3.094725,3.101402,5.217786,5.255942,4.775748,5.424529
23318,2.850000,2.935000,2.790000,2.795000,10200,0.0,0.0,ROMA,2025,6,...,3.498123,4.367188,4.171642,3.183371,3.140450,2.973404,5.026979,5.323810,4.737288,5.008960
23319,2.883000,2.900000,2.730000,2.790000,33600,0.0,0.0,ROMA,2025,6,...,3.671053,4.366197,4.292308,3.290094,3.671053,2.915360,4.894737,5.166666,4.860627,5.157116


5. **Determine the best holding period**:  
   - Find the number of months **(1 to 12)** where the **average (mean)** future growth is **maximal**.  
   - This optimal month shows an uplift of **>1%** compared to all others.  
   - Still, the average return remains **less than 1** (i.e., expected return is less than doubling your investment).


In [86]:
df1=stocks_df[stocks_df["Ticker"]!="NAKA"]
df2=df1[df1["Ticker"]!="SUPX"]


In [87]:
# 1. Select future growth columns
future_cols = [f'future_growth_{i}m' for i in range(1, 13)]

# 2. Filter out extreme outliers (e.g., > 10× return)
#filtered_df = future_growth_df[future_growth_df[future_cols].lt(10).all(axis=1)]

# 3. Compute average growth (mean) for each holding period
mean_growths = stocks_df[future_cols].mean()
mean_growths_sorted = mean_growths.sort_values(ascending=False)
print(mean_growths_sorted)


future_growth_12m    56.857467
future_growth_11m    56.231393
future_growth_10m    55.595600
future_growth_9m     54.764839
future_growth_8m     52.666066
future_growth_7m     50.616040
future_growth_6m     48.027666
future_growth_5m     45.484761
future_growth_4m     42.683224
future_growth_3m     37.650665
future_growth_2m     29.742199
future_growth_1m     17.575470
dtype: float64


---
### Question 4: [Strategy] Simple RSI-Based Trading Strategy


**What is the total profit (in $thousands$) you would have earned by investing $1000 every time a stock was oversold (RSI < 25)?**


---

#### Goal:
Apply a simple rule-based trading strategy using the **Relative Strength Index (RSI)** technical indicator to identify oversold signals and calculate profits.

---


#### Steps:

1. **Run the full notebook from Lecture 2 (33 stocks)**  
   - Ensure you can generate the merged DataFrame containing:  
     - OHLCV data  
     - Technical indicators  
     - Macro indicators  
   - Focus on getting **RSI** computed using **Code Snippets 8 and 9**.  
   - This process is essential and will help during the capstone project.



2. ⚠️ **IMPORTANT** Please use this file to solve the Home Assignment (**all next steps**)
 
   Download precomputed data using this snippet:

   ```python
   import gdown
   import pandas as pd

   file_id = "1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-"
   gdown.download(f"https://drive.google.com/uc?id={file_id}", "data.parquet", quiet=False)
   df = pd.read_parquet("data.parquet", engine="pyarrow")

K** (i.e., in thousands of dollars) that could be earned using this RSI-based oversold strategy from 2000–2025?

In [88]:
import gdown
import pandas as pd

file_id = "1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-"
gdown.download(f"https://drive.google.com/uc?id={file_id}", "data.parquet", quiet=False)
df = pd.read_parquet("data.parquet", engine="pyarrow")

Downloading...
From (original): https://drive.google.com/uc?id=1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-
From (redirected): https://drive.google.com/uc?id=1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-&confirm=t&uuid=5489a203-cc1a-4c94-9f45-1339cea87663
To: C:\Users\marco\Downloads\data.parquet
100%|███████████████████████████████████████████████████████████████████████████████| 130M/130M [00:08<00:00, 15.2MB/s]


3. **RSI Strategy Setup:**  
   - RSI is already available in the dataset as a field.  
   - The threshold for **oversold** is defined as `RSI < 25`.



In [89]:
df["rsi"]

0             NaN
1             NaN
2             NaN
3             NaN
4             NaN
          ...    
5686    68.312972
5687    67.123448
5688    67.563912
5689    68.218963
5690    69.670158
Name: rsi, Length: 229932, dtype: float64

4. **Filter the dataset by RSI and date:**  
   ```python
   rsi_threshold = 25
   selected_df = df[
       (df['rsi'] < rsi_threshold) &
       (df['Date'] >= '2000-01-01') &
       (df['Date'] <= '2025-06-01')
   ]


In [90]:
rsi_threshold = 25
selected_df = df[
       (df['rsi'] < rsi_threshold) &
       (df['Date'] >= '2000-01-01') &
       (df['Date'] <= '2025-06-01')
   ]
selected_df

,Open,High,Low,Close_x,Volume,Dividends,Stock Splits,Ticker,Year,Month,...,growth_brent_oil_7d,growth_brent_oil_30d,growth_brent_oil_90d,growth_brent_oil_365d,growth_btc_usd_1d,growth_btc_usd_3d,growth_btc_usd_7d,growth_btc_usd_30d,growth_btc_usd_90d,growth_btc_usd_365d
3668,20.056772,20.114241,19.405453,19.673643,99915200.0,0.0,0.0,MSFT,2000,2000-09-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3669,19.692798,19.807736,19.060636,19.309669,69037800.0,0.0,0.0,MSFT,2000,2000-09-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3680,18.256067,18.332693,17.317403,17.336559,85374000.0,0.0,0.0,MSFT,2000,2000-10-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3681,17.279087,17.336557,16.704395,16.991741,136453400.0,0.0,0.0,MSFT,2000,2000-10-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3682,17.010902,17.547282,16.934277,16.972589,81099400.0,0.0,0.0,MSFT,2000,2000-10-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4405,797.393417,806.609179,764.362578,769.289795,7251950.0,0.0,0.0,LT.NS,2020,2020-03-01,...,0.764911,0.515014,0.457056,0.336604,1.181878,1.234663,1.245515,0.610451,0.857647,1.514674
4406,774.673304,800.951963,758.522860,788.998840,6130185.0,0.0,0.0,LT.NS,2020,2020-03-01,...,0.753842,0.491171,0.431611,0.320580,1.001225,1.186226,1.114145,0.643468,0.862000,1.538415
4407,738.175133,739.041975,644.192390,660.662170,7308612.0,0.0,0.0,LT.NS,2020,2020-03-01,...,0.813666,0.496236,0.434706,0.322131,1.100520,1.035093,1.279557,0.663996,0.876243,1.595238
4408,698.550405,699.244329,621.756122,654.971985,7110384.0,10.0,0.0,LT.NS,2020,2020-03-01,...,0.802068,0.509668,0.437480,0.319412,1.049637,1.088881,1.288802,0.678603,0.925726,1.699390


5. **Calculate Net Profit Over 25 Years:**  
   - Total number of trades: **1568**  
   - For each trade, you invest **$1000**  
   - Use the 30-day forward return (`growth_future_30d`) to compute net earnings:  
     ```python
     net_income = 1000 * (selected_df['growth_future_30d'] - 1).sum()
     ```

   - **Final Answer:**  
     What is the **net income in $

In [91]:
net_income = 1000 * (selected_df['growth_future_30d'] - 1).sum()

# Answer 4

In [92]:
net_income_k = net_income / 1000
print(f"Net income over 25 years: ${net_income_k:.2f}K")


Net income over 25 years: $24.30K
